<a href="https://colab.research.google.com/github/YolaYing/zk-toolkit/blob/main/Plonk%2BKZG_Implementation(for_demo).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Develope Environment Preparation
Curve we used is BLS12-381
- Library we used: BLS21-381 curve implemented by Ethereum, using Python
- Lib link: https://github.com/ethereum/py_ecc/tree/main

You can use the following command to install all the packages we needed. Note that if you are using anaconda, package installation may be failed. Highly recommand using colab or some development friendly environment.

In [ ]:
!pip3 install py_ecc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 492.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.8 MB/s eta 0:00:00


# Preliminaries


## Form of Polynomial Representations(using baby parameters for demonstration)

A polynomial $\Phi(x) = \sum_{i = 0}^{n - 1} \phi_ix^i$ have two representation forms:
1. Coefficient Form
  - $\Phi(x)$ can be represented as a tuple of $n$ coefficients: $[\phi_0, \phi_1, ..., \phi_{n-1}]$
2. Evaluation Form
  - $\Phi(x)$ can be represented as a tuple of $n$ distinct evaluations: $[\Phi(x_0), \Phi(x_1), ..., \Phi(x_{n-1})]$
    - the set of values $\\{ x_0, x_1, ..., x_{n-1} \\}$ over which the polynomial is defined over is known as "evaluation domain"

In [ ]:
# For example, assume we have a polynomial Φ(𝑥)=4x^3+5x^2+3x+2 and some points on the polynomials
q = 17
poly_coefs = [2, 3, 5, 4]
evaluation_domain = [1, 4, 16, 13]
poly_evals = [14, 10, 0, 1]

print(f'coefficient form of phi(x) is {poly_coefs}')
print(f'evaluation form of phi(x) is {poly_evals}, defined over evaluation domain {evaluation_domain}')

coefficient form of phi(x) is [2, 3, 5, 4]
evaluation form of phi(x) is [14, 10, 0, 1], defined over evaluation domain [1, 4, 16, 13]


## Convert between Coefficient Form and Evaluation Form

### Defination
- **Fourier Transform**: convert from coefficient to evaluation
- **inverse Fourier Transform**: convert from evaluation to coefficient


### Naive Way
In naive way, each of those two transformation takes $O(n^2)$ computation
- **Fourier Transform**: evaluate $\Phi(x)$ at each $x_i$ in the evaluation domain
- **inverse Fourier Transform**: use *Lagrange Interpolation* to obtain unique degree $(n-1)$ polynomial passes through each of the $n$ points


### Optimized Way: Fast Fourier Transformation(FFT)
To improve the effectiveness of the transformation, we need to do the following steps:
- defined polynomials over finite field
  - restrict each coefficient $p_i \in F_q$ and each evaluation point $\Phi(x_i) \in F_q$
  - $q$ stands for curve order
- defined the evaluation domain as a multiplicative subgroup of $F_q$
  - evaluation domain is a set of "$n^{th}$ roots of unity", $\{ \omega^0, \omega^1, ..., \omega^{n-1} \}$ for some element $\omega \in F_q$ with order $n$(i.e. $\omega^n = 1$ mod $q$)
- implementation of FFT algorithm
  - if you are not familiar with FFT, highly recommend this video: https://www.youtube.com/watch?v=h7apO7q16V0

In [ ]:
from py_ecc.bls12_381 import curve_order
from py_ecc.fields.field_elements import FQ as Field

# construct a data structure F_q
class FQ(Field):
    field_modulus = curve_order
q = curve_order

# For example, assume we have a polynomial Φ(𝑥)=4x^3+5x^2+3x+2 and some points on the polynomials
poly_coefs = [FQ(2), FQ(3), FQ(5), FQ(4)]
poly_evals = [FQ(14), FQ(52435875175126190475982595682112313518914282969839895044333406231173219221502), FQ(0), FQ(3465144826073652318776269530687742778270252468765361963005)]

def find_nth_root_of_unity_realnum(n, q):
  '''
  Args:
  n: nth root of unity, which is the degree of polynomial
  q: finite field q

  Returns:
  omega: the nth root of unity, which is a element in finite field
  '''
  omega = FQ(5) ** ((q - 1) // n)
  return omega

omega = find_nth_root_of_unity_realnum(len(poly_coefs), q)
evaluation_domain = [omega**0, omega**1, omega**2, omega**3]

print(f'coefficient form of phi(x) is {poly_coefs}')
print(f'evaluation form of phi(x) is {poly_evals}, defined over evaluation domain {evaluation_domain}')

coefficient form of phi(x) is [2, 3, 5, 4]
evaluation form of phi(x) is [14, 52435875175126190475982595682112313518914282969839895044333406231173219221502, 0, 3465144826073652318776269530687742778270252468765361963005], defined over evaluation domain [1, 3465144826073652318776269530687742778270252468765361963008, 52435875175126190479447740508185965837690552500527637822603658699938581184512, 52435875175126190475982595682112313518914282969839895044333406231173219221505]


In [ ]:
import random

def FFT(poly_coefs, q):
  '''
  Args:
  poly_coefs: coefficient representation of the polynomial

  Returns:
  y: evaluation form of the polynomial
  '''
  # get the degree of the polynomial
  n = len(poly_coefs)
  if n == 1:
    return poly_coefs

  # in theory, omega should be the nth root of unity, which is a complex number
  # to use it in finite field, we use a algorithm here
  omega = find_nth_root_of_unity_realnum(n, q)
  # print(f'{n}^th of unity is {omega}')
  poly_coefs_e = poly_coefs[::2]
  poly_coefs_o = poly_coefs[1::2]
  y_e = FFT(poly_coefs_e, q)
  y_o = FFT(poly_coefs_o, q)
  y = [0] * n
  for j in range(int(n/2)):
    y[j] = y_e[j] + (omega**j)*y_o[j]
    y[j + int(n/2)] = y_e[j] - (omega**j)*y_o[j]
  return y

poly_evals = FFT(poly_coefs, q)
print(f'coefficient form of phi(x) is {poly_coefs}, after FFT, we can get the evaluation form of phi(x) is {poly_evals}')

coefficient form of phi(x) is [2, 3, 5, 4], after FFT, we can get the evaluation form of phi(x) is [14, 52435875175126190475982595682112313518914282969839895044333406231173219221502, 0, 3465144826073652318776269530687742778270252468765361963005]


### Inverse Fast Fourier Transformation(IFFT)
We have used FFT to achieve fast transformation from coefficient to evaluation. Now we will implement the inverse transformation, which is from evaluation to coefficient with slightly changing in the algorithm: update $\omega$ to $\frac{1}{n}\omega^{-1}$

detailed info: https://decentralizedthoughts.github.io/2023-09-01-FFT/#mjx-eqn-%5Cstar

In [ ]:
def IFFT(poly_evals, q):
  recursion_result = IFFT_recursion_part(poly_evals, q)
  # all elenent in result mutiply 1/n, which is inverse of n
  IFFT_final_result = [ x * (FQ(1)/FQ(len(poly_evals))) for x in recursion_result]
  return IFFT_final_result

def IFFT_recursion_part(poly_evals, q):
  '''
  Args:
  poly_coefs: coefficient representation of the polynomial
  lookup: pre-determined omega list

  Returns:
  y: evaluation form of the polynomial
  '''
  # get the degree of the polynomial
  n = len(poly_evals)
  if n == 1:
    return poly_evals

  # in theory, omega should be the nth root of unity, which is a complex number
  # to use it in finite field, we use a algorithm here
  omega = find_nth_root_of_unity_realnum(n, q)
  # inverse omega
  omega_inv = omega**(n-1)
  poly_evals_e = poly_evals[::2]
  poly_evals_o = poly_evals[1::2]
  y_e = IFFT_recursion_part(poly_evals_e, q)
  y_o = IFFT_recursion_part(poly_evals_o, q)
  y = [0] * n
  for j in range(int(n/2)):
    y[j] = int(y_e[j] + (omega_inv**j)*y_o[j])
    y[j + int(n/2)] = int(y_e[j] - (omega_inv**j)*y_o[j])
  return y

print(f'evaluation form of phi(x) is {poly_evals}, after IFFT(use exist omega lookup table), we can get the coefficient form of phi(x) is {IFFT(poly_evals, q)}')

evaluation form of phi(x) is [14, 52435875175126190475982595682112313518914282969839895044333406231173219221502, 0, 3465144826073652318776269530687742778270252468765361963005], after IFFT(use exist omega lookup table), we can get the coefficient form of phi(x) is [2, 3, 5, 4]


# KZG Implementation(Python Version)

The KZG Commitment Scheme is a commitment scheme that allows to commit to a polynomial $\Phi(x) = \phi_0 +\phi_1x+\phi_2x^2+...+\phi_lx^l$, where $\Phi(x) \in F_p[x]$ . 'to commit' means proving that you know the polynomial $\Phi(x)$ without revealing it.

The KZG commitment scheme consists of 4 steps:
1. Setup
2. Commit to Polynomials
3. Prove an Evaluation
4. Verify an Evaluation Proof

## Step 1: Setup

The first step is an one-time trusted setup and once it has done once, the following steps can be done repeatedly
1. Let $G_1$ and $G_2$ be pairing-friendly elliptic curve groups, determined by curve BLS12-381
2. Let $g_1$ be a generator of $G_1$ and $g_2$ be a generator of $G_2$
3. Let $l$ be the maximum degree of the polymonials we want to commit to ($l < p$)
4. Pick a random field element as secret parameter $\tau \in F_p$(usually done by MPC, to simplify, we just randomly choose one here)
5. Compute pp(public parameters, including proving key $pk$, and verifying key $vk$)$$pk = (g_1, g_1^\tau, g_1^{\tau^2},...,g_1^{\tau^l}), vk = g_2^\tau$$ and release it publicly
6. Discard secret parameter $\tau$ once the setup ceremony is done so that nobody can figure out its value

In [ ]:
from py_ecc.bls12_381 import G1, G2, Z1, multiply, add

# 1. Let G1,G2 be pairing-friendly elliptic curve groups, which are determined by the curve

# 2. Let g be a generator of G
g1 = G1
g2 = G2

# 3. Let l be the maximum degree of the polymonials, which is 16
l = 16

# 4. Pick a random field element from field F_p and p = curve order as secret parameter t
p = q
# t = 0 will lose all the security, so t cannot be 0
t = FQ(random.randint(1, p-1))
print(f'secret parameter 𝜏 is {t}')

# 5. Compute pp(public parameters)
def compute_public_parameters(g1, g2, t, l):

    pk = []
    accumulated = 1
    t_scalar = t
    for i in range(l + 1):
        # calculate g1, g1^t, g1^{t^2}...
        pk.append(multiply(g1, int(accumulated)))
        # calculate the exponential t, t^2, t^3, ...
        accumulated = accumulated * t_scalar
    vk = multiply(g2, int(t))
    return pk, vk

pk, vk = compute_public_parameters(g1, g2, t, l)
print(f'proving key pk: {pk}')
print(f'verifying key vk: {vk}')

secret parameter 𝜏 is 40434729172192096827009336261427730235302754744622279258050981829406623036311
proving key pk: [(3685416753713387016781088315183077757961620795782546409894578378688607592378376318836054947676345821548104185464507, 1339506544944476473020471379941921221584933875938349620426543736416511423956333506472724655353366534992391756441569), (1193244812650916127625064910500992265663422853459628004033737085594018331009212382341516068390371104378007330986118, 1109142381163146218466848879784830646546808045773662144060749927391990490535698641398731805623828985596234878463532), (1052018718780865389222790804074898640995670174234093816540110555110800906231323268196260438212562973245470708425601, 1731865486767691263458193795349951397277561212010516044988305523161542649511683946131999433428453791690747940088706), (357283692228559460687608554499824510775545394970475493277430899946385550387630984723518054425242372173458063205397, 2436323255616352838586647451251825421129330371644143033950

## Step 2: Commit to Polynomials
In reality, we arithmetize circuits and use Plonkish to get polynomials in this step
1. Given a polynomial $\Phi(x) = \sum_{i=0}^l \phi_i x^i$
2. Compute and output commitment $c = g^{\Phi(\tau)}$
   - Wait! $\tau$ has already been discard right? How can committer compute $\tau$?
   - Although he cannot compute $\Phi(\tau)$ directly, he can use public parameters to help with it:
$$\prod_{i=0}^l(g^{\tau^i})^{\phi_i} = g^{\sum_{i=0}^l \phi_i \tau^i} = g^{\Phi(\tau)}$$

In [ ]:
# assume we have a polynomial Φ(𝑥)=4x^3+5x^2+3x+2 or some points on the polynomials
# we can use FFT or IFFT to do the transformation between the two forms
poly_coefs = [FQ(2), FQ(3), FQ(5), FQ(4)]

In [ ]:
# compute commitment of the polynomial
def poly_commitment(pk, g1, poly_coefs):

  com = Z1
  for i in range(len(poly_coefs)):
    com = add(multiply(pk[i], int(poly_coefs[i])), com)
  return com

com = poly_commitment(pk, g1, poly_coefs)
print(f'commitment of polynomial is {com}')

commitment of polynomial is (2334721901160606975747982670666481410862613451440319525655847840343065350187448491998696068597460962188520111694861, 1165031092905317903788628866560570156024045456410458593603239499718406550985163969759858573198526426255495325082100)


## Step 3: Prove an Evaluation
In this period, the Verifier will ask Prover to 'OPEN' the commitment $c$ to a random specific point $a \in F_p$, in other word, Prover have to evaluation $\Phi(x)$ and commit the result in the form of opening triplet $OT = (a, b, \pi)$
1. Given an evaluation $\Phi(a) = b$
2. Compute and output proof of the evaluation $\pi = g_1^{q(\tau)}$, where $q(x) := \frac{\Phi(x)-b}{x-a}$
    - $q(x)$ is quotient polynomial: if $\Phi(a) = b$, that means $a$ is a root of $\Phi(x) - b$
    - so $\Phi(x) - b$ can be expressed as $\Phi(x) - b = q(x)(x-a)$, $q(x)$ is a polynomial
    - on the other hand, $q(x)$ exists if and only if $\Phi(a) = b$, so the existence of this quotient polynomial therefore serves as a proof of the evaluation

In [ ]:
# Verifier first choose the random point a
def evaluation_poly(a, poly_coefs):
  b = FQ(0)
  for i in range(len(poly_coefs)):
    b += (a**i)*(poly_coefs[i])
  return b

# according to defination
def random_generate_a(poly_coefs, p):
  # create a list of omega
  n = len(poly_coefs)
  omega = find_nth_root_of_unity_realnum(n, p)
  omega_list = []
  for i in range(n):
    omega_list.append(omega**i)

  # a cannot in omega list, otherwise x-a will be 0
  a = FQ(random.randint(0, p))
  while a in omega_list:
    a = FQ(random.randint(0, p))

  return a

a = random_generate_a(poly_coefs, p)
b = evaluation_poly(a, poly_coefs)
print(f'random element we chosen is {a}, the evaluation b is {b}')

random element we chosen is 22814109766336013075561986125880379584857082925698224031213951947309399302381, the evaluation b is 45642593918252489443187268940545354309014294680099687017493847768364997103854


In [ ]:
# compute q(x)
# 1. if poly in coefficient form, convert poly_coefs to evals
# 2. calculate quotient polynomial in evaluation form
# 3. convert quotient poly evals to coefs if needed

def quotient_poly(a, b, poly_evals, p):
  '''Args:
  a: randomly sampled field element
  b: evaluation on a
  poly_evals: input of poly need to be in evaluation form

  Returns:
  q_poly_evals: q poly in evaluation form
  '''
  q_poly_evals = []
  n = len(poly_evals)
  omega = find_nth_root_of_unity_realnum(n, p)

  for i in range(len(poly_evals)):
    q_poly_evals.append((poly_evals[i]-b)/(omega**i-a))

  return q_poly_evals

q_poly_evals = quotient_poly(a, b, poly_evals, p)
print(f'quotient polynomial in evaluation form is {q_poly_evals}')

quotient polynomial in evaluation form is [27783391858465084461048781107638308487870880463626264088707592488555803653841, 25167300037570457153357809204363066822160557950018254199173346563744749476850, 2578139253155551294896113625153169322085874559623385306806953009896352788322, 5194231074050178602587085528428410987796197073231395196341198934707406965297]


In [ ]:
# convert quotient polynomial from evaluation form to coefficient form
q_poly_coefs = IFFT(q_poly_evals, q)
print(f'quotient polynomial in coefficient form is {q_poly_coefs}')

quotient polynomial in coefficient form is [41398703143373413117696317620488721823823653761888643609059102099195368813334, 38820563890217861822800203995335552501737779202265258302252149089299016025016, 4, 0]


In [ ]:
# compute proof of the evaluation pi
pi = poly_commitment(pk, g1,  q_poly_coefs)
print(f'proof of the evaluation pi is {pi}')

proof of the evaluation pi is (1050066443485827318585387568702081403177548750155307357050157496204472296835137754305294166981923702975997335923568, 1941453761804587589505945531522177555326690381186979584739056451681900711762666290390823478163534514420569158511889)


## Step 4: Verify an Evaluation Proof
1. Given a commitment $c = g^{\Phi(\tau)}$, and an evaluation $\Phi(a) = b$, and a proof $\pi = g^{q(\tau)}$
2. Verify that $e(\frac{c}{g^b}, g) = e(\pi,\frac{g^\tau}{g^a})$, where $e$ is a non-trivial bilinear mapping
    - the purpose of verification: $\Phi(x) - b = q(x)(x-a)$, checking this equality holds for $x = \tau$
    - according to the definition of bilinear mapping, it is equivalent to: $e(g_1, g_2)^{\Phi(\tau) - b} = e(g_1, g_2)^{q(\tau)(\tau-a)}$, that is $e(g_1^{\Phi(\tau) - b}, g_2) = e(g_1^{q(\tau)}, g_2^{\tau-a})$
    - that is $e(com-g_1^b, g_2) = e(\pi, vk - g_2^a)$

In [ ]:
from py_ecc.bls12_381 import pairing, neg

# now it is time to do the varification
print(f'result of verification: {pairing(g2, add(com, neg(multiply(g1, int(b))))) == pairing(add(vk, neg(multiply(g2, int(a)))), pi)}')

result of verification: True


# Plonk Implementation(Python Version)

## Problem Definition

We take **Square-Fibonacci** as an example to demonstrate the process of proof generation

Defination of Square-Fibonacci Problem:
- Let $f_0 = 1, f_1 = 1$
- For $i \ge 2$, define $f_i:=(f_{i-2})^2+(f_{i-1})^2 \ mod \ q$
    - $q$ is a large prime integer, used to bound the size of each element, so that it can be represented by some predetermined number of bits.

Let $n$ be some very large integer. For convenience, we assume $n$ is a power of 2

Let $k$ be the $n^{th}$ Square-Fibonacci number

**Our goal**: generate an efficiently-verifiable proof $\pi$ showing that indeed $k$ is the $n^{th}$ Square-Fibonacci number(i.e. $f_n = k$)

## Phases of Proof Generation
The Plonk-based proof generation consists of 3 steps:
1. Filling in the trace table
2. Committing to the trace table
3. Proving the trace table's correctness

In [ ]:
# some basic statement
# q here is equialent to the p above
q = curve_order

# assume we hope to prove '8th Square-Fibonacci number is k'
n = 8
k = FQ(317754178345286893212434)

# according to defination, f(0) = 1, f(1) = 1
f_0 = FQ(1)
f_1 = FQ(1)

## Step 1: Filling in the Trace Table
The trace table is a 2-dimensional matrix where 'witness' or 'trace' is written down, that is (n rows * 5 cols)
- 5 columns:
    - $A, B, C$: represent witness data / private input, each row lists 3 sequential Sequare-Fibonacci numbers
        - e.g. the $i^{th}$ row $(f_i, f_{i+1}, f_{i+2})$ is a witness for $(i+2)^{th}$ Sequare-Fibonacci number
    - $S$: represents selector column, indicating a certain mathmatical relation should hold over the element of the row.
        - $1$ represents the first 3 elements of the row $(a, b, c)$ must satisfy $c = a^2 + b^2 \ mod \ q$
        - $0$ represents the condition does not need to be satisfied.
    - $P$: represents public inputs, inputs to the circuit that are public known.
        - e.g. the first two values of the sequence $f_0, f_1$ and $k$ as the value to be proved
- n rows: left a blank row, so that the height of the table becomes $n$, an even power of 2
    - $1^{st}$ row: $f_0, f_1, f_2, 1, f_0$
    - $2^{nd}$ row: $f_1, f_2, f_3, 1, f_1$
    - $3^{rd}$ row: $f_2, f_3, f_4, 1, k$
    - ...
    - $(n-2)^{th}$ row: $f_{n-2}, f_{n-1}, f_n, 1, "" $
    - $(n-1)^{th}$ row: $"", "", "", 0, ""$

Next step is to fill in the trace table: either copy or compute over $F_q$

In [ ]:
# generate witness/fill in the trace table
def witness_generation(f_0, f_1, k, n):

    trace_table = []

    # init col A, B, C, S
    f_a = f_0
    f_b = f_1
    f_c = f_b
    S = FQ(1)
    placeholder = FQ(0)

    for i in range(n-1):
        f_a = f_b
        f_b = f_c
        f_c = f_a**2 + f_b**2
        trace_table.append([f_a, f_b, f_c, S, placeholder])

    # add a blank row to get n row
    S = FQ(0)
    trace_table.append([placeholder, placeholder, placeholder, S, placeholder])

    # add public parameters
    trace_table[0][4] = f_0
    trace_table[1][4] = f_1
    trace_table[2][4] = k

    return trace_table

trace_table = witness_generation(f_0, f_1, k, n)

print(f"trace table is: {trace_table}")

trace table is: [[1, 1, 2, 1, 1], [1, 2, 5, 1, 1], [2, 5, 29, 1, 317754178345286893212434], [5, 29, 866, 1, 0], [29, 866, 750797, 1, 0], [866, 750797, 563696885165, 1, 0], [750797, 563696885165, 317754178345286893212434, 1, 0], [0, 0, 0, 0, 0]]


## Step 2: Commit to the Trace Table

### interpret the trace table columns as polynomials

Each column can be considered as a length-$n$ vector of finite field elements $\rightarrow$ this vector can be regarded as the evaluation form of a polynomial $A(x)$ with degree $(n-1)$: the $i^{th}$ element of $A$ corresponds to the evaluation $A(\omega^i)$, where $\omega \in F_q$ is **$n^{th}$ root of unity** and has order $n$

In [ ]:
# inverse trace table and get the evaluation form of column polynomials
inverse_trace_table = list(zip(*trace_table))
inverse_trace_table

[(1, 1, 2, 5, 29, 866, 750797, 0),
 (1, 2, 5, 29, 866, 750797, 563696885165, 0),
 (2, 5, 29, 866, 750797, 563696885165, 317754178345286893212434, 0),
 (1, 1, 1, 1, 1, 1, 1, 0),
 (1, 1, 317754178345286893212434, 0, 0, 0, 0, 0)]

### Commit to column polynomials
Now that we have known how to interpret the columns as polynomials, we can commit to each of them using a polynomial commitment scheme.

In [ ]:
com_list = []
col_poly_coefs = []
for i in range(len(inverse_trace_table)):
    col_poly_coef = IFFT(inverse_trace_table[i], p)
    col_poly_coefs.append(col_poly_coef)
    com = poly_commitment(pk, g1, col_poly_coef)
    com_list.append(com)
print(com_list)

[(2121554138867427275116034715986226759372513147535533209207100626223741766609870436529367490430403404651398593127669, 789918926296013400617168934331301652695241462754118994262568520624083999672254706565865450308512748558999392662595), (3999169330449091235659967038513062786029816328438282384121736373492240313307176622833263179319322826361019880261177, 549976911676065028758314832410527527668266507874322356699501941064313575185452633158194355224675554243211861563999), (1529891052057887133138842414484662132124959504346936447784607572440362254364592541183693906292754198547315905779340, 2708992316720554983134563242916186758736539285621173689042306959356318014038582470507021766642744894651794391164024), (414218354840137055237867557860020569217464651437712719024622635496832612807252853092095081989370412479584520420776, 953026705346469508196605634396578626214789062322910758633472353121470827036336288604731588053569079804932623039959), (1946953203049277147217555238176753581381417980475536594862

## Step 3: Proving the Trace Table's Correctness
### Define the constraints of the trace table
In order to ensure the original trace table to be valid, we should have the following constraints:
- Square-Fibonacci constraints:
    - each selected row $i$'s first three elements $(a,b,c)$ must satisfy $c_i = a_i^2 + b_i^2 \ mod \ q$
- Wiring constraints:
    - for consecutive rows with value $[a_i, b_i, c_i]$ and $[a_{i+1}, b_{i+1}, c_{i+1}]$, we require $a_{i+1} = b_i$ and $b_{i+1} = c_i$
- Public input constraints:
    - the first row must start with the first two Square-Fibonacci numbers: $a_0 = p_0, b_0 = p_1$
    - the $n^{th}$ Square-Fibonacci must match the claimed result value: $c_{n-2} = p_2$

The above constraints can be represented by one or more relations between the column polynomials. For example, Square-Fibonaci constraints can be expressed as $S(x)·(A(x)^2 + B(x)^2 - C(x))=0, for\ all\ x\in \{w^0, w^1, ..., w^{n-1}\}$. For shorten, we will label left-hand side $\phi_0(x):=S(x)·(A(x)^2 + B(x)^2 - C(x))$  All our constraints can be expressed as $\phi_i(x) = 0,\ for\ all\ x\in \{w^0, w^1, ..., w^{n-1}\}$

### Square-Fibonaci Constraints

It is equivalent to **Gate Constraint** in vanilla plonk:

Obviously, $a^2+b^2 = c$ can be seen as the combination of 2 multiplication gates and 1 addition gate. To simplify, we take those as a whole thing and just use one line of record to represent it.

In [ ]:
def square_fib_constraint_poly(inverse_trace_table, p):

  a_poly_coefs = IFFT(inverse_trace_table[0], p)
  b_poly_coefs = IFFT(inverse_trace_table[1], p)
  c_poly_coefs = IFFT(inverse_trace_table[2], p)
  s_poly_coefs = IFFT(inverse_trace_table[3], p)

  # construct a polynomial phi_0(x) = S(x)(A(x)^2+B(x)^2-C(x))
  phi_0_coefs = mul_poly(s_poly_coefs,sub_poly(add_poly(mul_poly(a_poly_coefs, a_poly_coefs), mul_poly(b_poly_coefs, b_poly_coefs)),c_poly_coefs))

  return phi_0_coefs


def mul_poly(poly1_coefs, poly2_coefs):
  multiply_poly = [0]*(len(poly1_coefs)+len(poly2_coefs)-1)
  for o1,i1 in enumerate(poly1_coefs):
    for o2,i2 in enumerate(poly2_coefs):
      multiply_poly[o1+o2] += i1*i2
  return multiply_poly

def add_poly(poly1_coefs, poly2_coefs):

  if len(poly1_coefs) >= len(poly2_coefs):
    addition_poly = poly1_coefs
    min_length = len(poly2_coefs)
  else:
    addition_poly = poly2_coefs
    min_length = len(poly1_coefs)

  for i in range(min_length):
    addition_poly[i] = poly1_coefs[i] + poly2_coefs[i]

  return addition_poly

def sub_poly(poly1_coefs, poly2_coefs):

  if len(poly1_coefs) >= len(poly2_coefs):
    subtraction_poly = poly1_coefs
    min_length = len(poly2_coefs)
  else:
    subtraction_poly = poly2_coefs
    min_length = len(poly1_coefs)

  for i in range(min_length):
    subtraction_poly[i] = poly1_coefs[i] - poly2_coefs[i]

  return subtraction_poly

square_fib_constraint_poly(inverse_trace_table, p)

[9217243683127650670215423136204564307406542431733373805594493117736508689447,
 6666845327329705532963430625265951311576613171241245242132100072955433688671,
 44267534341411372821110843579994475498172624576994019134899687329660324137538,
 27734915317295573979553823553557321860270268471208809659398702095994161952080,
 37702370013672502632704303140624342804248711303899845502198214997157027878207,
 11583562288821433247490937116774158461617493607772389006562557827980992423716,
 19431544404868491752428433057080910022889750308878986243969634670562912518450,
 12399440026714068747233115926566936096429177141939361413989904821957360748254,
 20041303549030907216090313199445756936739800542200228711060022339667450807871,
 1640433480202112741396001466119286110343076964539974284673711265395784145821,
 13311292830147122275312324513496984175656020478185536494792425391434485506145,
 41059692124944929433452274305014365636083572356756835655701111647528539392349,
 487287176898972862051909273570054856936489

### Wiring Constraints
we can also use other ways to represent wiring constraints

In [ ]:
# another way to implement wiring constraints
# using Fi_1 to represent a_i+1 = b_i
def wiring_constraint_poly(inverse_trace_table, p):

  a_poly_coefs = IFFT(list(inverse_trace_table[0][1:])+[FQ(0)], p)
  b_poly_coefs = IFFT(inverse_trace_table[1], p)
  s_poly_coefs = IFFT(list(inverse_trace_table[3][1:])+[FQ(0)], p)

  # construct a polynomial phi_0(x) = S(x+1)(A(x+1)-B(x))
  phi_1_coefs = mul_poly(s_poly_coefs, sub_poly(a_poly_coefs, b_poly_coefs))

  return phi_1_coefs

wiring_constraint_poly(inverse_trace_table, p)

[11470347694558854167379193236165680026994808359490420773694550340558718051128,
 31658831632200537539783533621391251008881967401427553257134119658570500984004,
 9251157176012360485135106648195872832425943262155855154068869944562980101270,
 4629573589425733109598056930899130194516745852141088748221668437883354628737,
 27092786656927993916917219376235083497636322707752077888330974664656646497822,
 16147469831419210761107806895310410948843277724729960615923746116523503448252,
 47225732227417011674567510410490426723757052077807989737374736443825102691387,
 0,
 40965527480567336312068547272020285810695744141037217048909108359379863133385,
 20777043542925652939664206886794714828808585099100084565469539041368080200509,
 43184717999113829994312633859990093005264609238371782668534788755375601083243,
 47806301585700457369849683577286835643173806648386549074381990262055226555776,
 25343088518198196562530521131950882340054229792775559934272684035281934686691,
 3628840534370697971833993361287555488

In [ ]:
# wiring constraints
# using Fi_2 to represent b_i+1 = c_i
def wiring_constraint_poly2(inverse_trace_table, p):

  b_poly_coefs = IFFT(list(inverse_trace_table[1][1:])+[FQ(0)], p)
  c_poly_coefs = IFFT(inverse_trace_table[2], p)
  s_poly_coefs = IFFT(list(inverse_trace_table[3][1:])+[FQ(0)], p)

  # construct a polynomial phi_0(x) = S(x+1)(A(x+1)-B(x))
  phi_2_coefs = mul_poly(s_poly_coefs, sub_poly(b_poly_coefs, c_poly_coefs))

  return phi_2_coefs

wiring_constraint_poly2(inverse_trace_table, p)

[36049664182899255954620321599377851513412254844112750973250561136337128325687,
 26478744040510257060869321766053525138872800379087851350621017645042397135506,
 32410835611437218704466690526562532753646862728670419077503224720809139370953,
 11748757413644117521565380048665272397719182982198598038819999551729010043282,
 5425566726670899114035402883671431159983196801318069536656707248542603833947,
 45997037633194194370717268143927422353966899878804396217539101952889432168702,
 37540168226135780995704033570283361978050109689519990765762645485100182660726,
 0,
 16386210992226934524827418908808114324278297656414886849353097563601452858826,
 25957131134615933418578418742132440698817752121439786471982641054896184049007,
 20025039563688971774981049981623433084043689771857218745100433979129441813560,
 40687117761482072957882360459520693439971369518329039783783659148209571141231,
 47010308448455291365412337624514534677707355699209568285946951451395977350566,
 643883754193199610873047236425854348

In [ ]:
constraints_poly = []

# Square-Fibonaci constraints
constraints_poly.append(square_fib_constraint_poly(inverse_trace_table, p))

# using Fi_1 to represent a_i+1 = b_i
constraints_poly.append(wiring_constraint_poly(inverse_trace_table, p))

# using Fi_2 to represent b_i+1 = c_i
constraints_poly.append(wiring_constraint_poly2(inverse_trace_table, p))

constraints_poly

[[9217243683127650670215423136204564307406542431733373805594493117736508689447,
  6666845327329705532963430625265951311576613171241245242132100072955433688671,
  44267534341411372821110843579994475498172624576994019134899687329660324137538,
  27734915317295573979553823553557321860270268471208809659398702095994161952080,
  37702370013672502632704303140624342804248711303899845502198214997157027878207,
  11583562288821433247490937116774158461617493607772389006562557827980992423716,
  19431544404868491752428433057080910022889750308878986243969634670562912518450,
  12399440026714068747233115926566936096429177141939361413989904821957360748254,
  20041303549030907216090313199445756936739800542200228711060022339667450807871,
  1640433480202112741396001466119286110343076964539974284673711265395784145821,
  13311292830147122275312324513496984175656020478185536494792425391434485506145,
  41059692124944929433452274305014365636083572356756835655701111647528539392349,
  48728717689897286205190927357

### Combine constraints

- **Naive way to proof the contraints**

    In general, we have $m$ constraint polynomials $\phi_0(x), \phi_1(x), ..., \phi_{m-1}(x)$. Sample a random field element $\gamma \in F_q$, and then take a random linear combination of the individual constraints:$$\phi(x) := \gamma^0·\phi_0(x) + \gamma^1·\phi_1(x)+...+\gamma^{m-1}·\phi_{m-1}(x)$$
    and we need the constraints satisfied at every row, that is $\phi(\omega^i) = 0 \ for \ all\ 0\le i <n$, in this case we need $n$ evaluation proofs


- **Using quotient polynomial**

    we can prove such constraint $\phi(x)$ using only one evaluation proof:
    - quotient polynomial:
        \begin{aligned}
        \phi(\omega^i) = 0 \ for \ all\ 0\le i <n\ &{\Leftrightarrow}\ (x-\omega^i)|\phi(x)\ for \ all\ 0\le i<n\\
        &{\Leftrightarrow}\ \prod^{n-1}_{i=0}(x-\omega^i)|\phi(x)\ for \ all\ 0\le i <n\\
        &{\Leftrightarrow}\ (x^n-1)|\phi(x)\\
        &{\Leftrightarrow}\ \exists Q(x)\ s.t.\phi(x)=Q(x)·(x^n-1)
        \end{aligned}
      now we just need to prove there exists a polynomial Q(x)
    - compute the quotient polynomial:
        $$
        Q(x):= {\phi(x) \over {x^n-1}} = {{\gamma^0·\phi_0(x) + \gamma^1·\phi_1(x)+...+\gamma^{m-1}·\phi_{m-1}(x)} \over {x^n-1}}
        $$
      degree of $Q(x)$ is $2n-3$, so we need at least $2n-2$ evaluation points to represent it
      
      we make it a round number and use $2n$ evaluation points, our previous evaluation domain do not work anymore, because the order of $\omega$ is only $n$. We therefore need to pick some other element $\beta \in F_q$ with order $2n$. Then we evaluate $Q(x)$ over the evaluation domain $\{\beta^0, \beta^1,...,\beta^{2n-1}\}$

In [ ]:
# define Phi(x)
def Phi(constraints_poly, n, p):
  # randomly gerenate gamma in F_p
  gamma_list = []
  Phi_poly_coefs = [0] * n

  for i in range(len(constraints_poly)):
      gamma = FQ(random.randint(0, p))
      gamma_list.append(gamma)
      phi_i = [gamma * constraints_poly[i][j] for j in range(len(constraints_poly[i]))]
      Phi_poly_coefs = add_poly(phi_i, Phi_poly_coefs)

  return Phi_poly_coefs, gamma_list

Phi_poly_coefs, gamma_list = Phi(constraints_poly, len(constraints_poly[0]), p)
print(f'the combination of contraints is {Phi_poly_coefs}')
print(f'the ramdom generated list of gamma is {gamma_list}')

the combination of contraints is [49055081158131710478159534583320415848715959091203751514703764679003849589182, 23527014410647066005681357424018502058179497965147260126029509417815060102830, 32049647844870507614243143813268006808968701494297911963173064860417692417655, 32035202929027505587200093771003462574029379846138339802431977330709640447833, 41350328662722042609251686581931858554205475205588143577705037090674675092978, 24477969498297646806829944489297171250668698147360246098503300266867552509222, 37043683659952007455302770020656193595890851968719157052951825962706800437167, 18916579483817845244490973913489027663137726832261248716252583436098262783142, 25262640056214501318479400369897723060396573889467389680151247442166363530491, 38127451411460775309331072764872637864584632775803767218616499083830433195948, 50354196917678888239284038520950165149178332575361412600990308435762782879716, 5631673560366180798838929090703727937113441871629967000252670816402602940947, 3042385851047609478

In [ ]:
def Quotient_poly(Phi_poly_coefs, n, p):
  '''Args:
  a: randomly sampled field element
  b: evaluation on a
  poly_evals: input of poly need to be in evaluation form

  Returns:
  Q_poly_evals: q poly in evaluation form
  '''
  Q_poly_evals = []

  omega = find_nth_root_of_unity_realnum(2*n, p)

  for i in range(2*n):

    Phi_poly_eval = evaluation_poly(omega**i, Phi_poly_coefs)
    Q_poly_evals.append(Phi_poly_eval/((omega**i)**n-FQ(1)))

  return Q_poly_evals

Q_poly_evals = Quotient_poly(Phi_poly_coefs, n, p)
print(f'quotient polynomial in evaluation form is {Q_poly_evals}')

quotient polynomial in evaluation form is [0, 6627971511873201344658318099783262099490928244267071618705636954132811605239, 0, 45737554521941954955172828037063424484164601737456281858870453595174026430255, 0, 45020880976460281360609718086033931983297927069838162635146840028567634906859, 0, 34630370478999413702862233076220038580487767130351459498046569448374637263073, 0, 44802542530179667582610298771195296663974132924770727605652706279821960165035, 0, 21664484464847581645682765703468010196541230858867972135739395479134949604395, 0, 50633481310488534025081651047010523546285949665729890840802787703685014920655, 0, 5419709830051566410605130646593262766620605984985189070849248748378454532930]


### Committing to the quotient polynomial
Now we have get the evaluation form of $Q(x)$, we can compute its commitment. Note, degree of $Q(x)$ is larger than the column polynomials, so it requires a larger KZG setup

In [ ]:
Q_poly_coefs = IFFT(Q_poly_evals, p)
Q_poly_coefs

[38849257615670345898963570439041844449043563194997513751377453071052472357502,
 19063725705730387654665536382436318932292316387901883609308249541915216597974,
 25177098458839444119642019260475082574589166287680706300495154217881391439858,
 29033774367746185639143334799444846887401997186078802411428164758170592062730,
 41429866842801142633487450657224713940866724768070705367021256258467952371051,
 30448232808238631938862785381845611318195434069570813649895574517199795394629,
 7696095757587091512072485243764886120899850265904240384825916368615890373673,
 42977585433217267857202253551441452006121689084397013464477366981889449792942,
 13586617559455844580484170069144121388646989305530124071226205628886108827011,
 33372149469395802824782204125749646905398236112625754213295409158023364586539,
 27258776716286746359805721247710883263101386212846931522108504482057189744655,
 23402100807380004840304405708741118950288555314448835411175493941767989121783,
 1100600833232504784596028985096125189682

In [ ]:
Q_com = poly_commitment(pk, g1, Q_poly_coefs)
print(f'commitment of polynomial is {Q_com}')

commitment of polynomial is (1466360593450773475302590639406302389267345026270393868612991659540778742557858339414802158314745730358739442866498, 2671804589350221450321678338770721699533040077057819055766446370041956202640328871034736906612976321500336617060521)


### Proving an Evaluation
Now we have committed all column polynomials from the trace table and have also committed to the quotient polynomial. Now the prover needs to demonstrate the quotient polynimial really exist and it was computed correctly.

This can be achieve through the following steps:
1. sample a random point $\alpha \in F_q$
2. generate and output KZG proofs for all column polynimials and the quotient polynomials at point $\alpha$ $$ Q(\alpha):= {\phi(\alpha) \over {\alpha^n-1}} = {{\gamma^0·\phi_0(\alpha) + \gamma^1·\phi_1(\alpha)+...+\gamma^{m-1}·\phi_{m-1}(\alpha)} \over {\alpha^n-1}}$$
$\alpha$ is sampled at random, so the property holds at $\alpha$, then it holds everywhere

In [ ]:
# Verifier first choose the random point a
a = FQ(random.randint(0, p))
print(f'random point a is {a}')

# calculate evaluation for all polynomials, and proof of the evaluation pi
## for column polynomials
col_evaluation = []
col_q_poly_evals = []
col_pi = []
for poly_coef in col_poly_coefs:
    # compute evaluation at a
    b = evaluation_poly(a, poly_coef)
    col_evaluation.append(b)
    # compute q(x)
    poly_eval = FFT(poly_coef, p)
    q_poly_evals = quotient_poly(a, b, poly_eval, p)
    col_q_poly_evals.append(q_poly_evals)
    # compute proof of the evaluation pi
    q_poly_coef = IFFT(q_poly_evals, p)
    pi = poly_commitment(pk, g1,  q_poly_coef)
    col_pi.append(pi)
print(f'evalutions for column polynomials are {col_evaluation}, and proof of evaluations are {col_pi}')

## for quotient polynomials
# compute evaluation at a
Q_evaluation = evaluation_poly(a, Q_poly_coefs)
# compute q(x)
Q_q_poly_eval = quotient_poly(a, Q_evaluation, Q_poly_evals, p)
# compute proof of the evaluation pi
Q_q_poly_coef = IFFT(Q_q_poly_eval, p)
Q_pi = poly_commitment(pk, g1, Q_q_poly_coef)
print(f'evaluation for quotient polynomial is {Q_evaluation},and proof of evaluations are {Q_pi}')

random point a is 42670908556048958175010494124117913770971306361356432100176861913057641969324
evalutions for column polynomials are [30418493975628262728068751303937601687741293623803608325590699908007437471541, 21568437621093640336972282553260726424706841888148821894574946940754614488771, 52306808783501808116517638893177859254254384536085666720651889586788967893853, 34355317477759176958102565752339184237102581669158749011299453207830791263573, 48321594924163810925743031796475461234771864055930307382453393957781671469157], and proof of evaluations are [(3863687297288056301486213638584767290969095583228892753520897385690301780482953392284164228048494544731968486118042, 1959410625719638613969227346943717382929460153280219286314395193916737311338193717642214637718888133645187208634133), (3254761686763006683617835440881604909111446876917007228348159171430701210495066574741194437694317035867381657220493, 2023854375722711664962880238084801315540376985716373390974369609724720771040405371293

### Verification
the verifier need to check two things:
1. each evaluation proof is correct
2. the quotient polynomial holds at point $\alpha$

In [ ]:
# verify each evaluation proof is correct
for i in range(len(com_list)):
    # print(f'verification result of column {i} : {evaluation_verification(com_list[i], col_evaluation[i], a, col_pi[i], g1, g2, vk)}')
    print(f'verification result of column {i} : {pairing(g2, add(com_list[i], neg(multiply(g1, int(col_evaluation[i]))))) == pairing(add(vk, neg(multiply(g2, int(a)))), col_pi[i])}')

# verify the quotient polynomial holds at point a
# assert GT.pairing(Q_com - g1*Scalar(Q_evaluation), g2) == GT.pairing(Q_pi, vk - g2*Scalar(a))
# print(f'verification result:{evaluation_verification(Q_com, Q_evaluation, a, Q_pi, g1, g2, vk)}')
print(f'verification result of quotient polynomial: {pairing(g2, add(Q_com, neg(multiply(g1, int(Q_evaluation))))) == pairing(add(vk, neg(multiply(g2, int(a)))), Q_pi)}')

verification result of column 0 : True
verification result of column 1 : True
verification result of column 2 : True
verification result of column 3 : True
verification result of column 4 : True
verification result of quotient polynomial: True


### *Wiring Constraints(from vanilla plonk)

The purpose of wiring constraints is to connect the sea of gates we get in the last step by proving certain gates' inputs/outputs equal each other.

To construct wiring contraints polynomials, we need to following steps:
1. Verifier choose random $\beta, \gamma \in F_p$ and sends them to Prover
2. Construct two matrix:
  - matrix of indexes for original computation trace table
    - we use roots of unity to represent the indexes:
      - for column A: $\omega^0, \omega^1, \omega^2, \omega^3, ..., \omega^{n-1}$
      - for column B: $2\omega^0, 2\omega^1, 2\omega^2, 2\omega^3, ..., 2\omega^{n-1}$
      - for column C: $3\omega^0, 3\omega^1, 3\omega^2, 3\omega^3, ..., 3\omega^{n-1}$
  - matrix of indexes after permutation*
    - permuation means:
      - assume we have three positions in the table share the same value:
        - the $a$ column in the $4^{th}$ line: (4, a), index = $\omega^3$
        - the $a$ column in the $7^{th}$ line: (7, a), index = $\omega^6$
        - the $c$ column in the $2^{rd}$ line: (2, c), index = $3\omega^1$
      - then we store:
        - the position (7, a) store (4, a)'s index: $\omega^3$
        - the position (2, c) store (7, a)'s index: $\omega^6$
        - the position (4, a) store (2, c)'s index: $3\omega^1$
    - in our case:
      - matrix before permutation
        - for column A: $\omega^0, \omega^1, \omega^2, \omega^3, ..., \omega^{n-1}$
        - for column B: $2\omega^0, 2\omega^1, 2\omega^2, 2\omega^3, ..., 2\omega^{n-1}$
        - for column C: $3\omega^0, 3\omega^1, 3\omega^2, 3\omega^3, ..., 3\omega^{n-1}$
      - matrix after permutation(constraints: $a_{i+1} = b_i$ and $b_{i+1} = c_i$)
        - for column A: $\omega^0, 2\omega^0, 2\omega^1, 2\omega^2, ..., 2\omega^{n-2}, 2\omega^{n-1}$
        - for column B: $\omega^1, 3\omega^0, 3\omega^1, 3\omega^2, ..., 3\omega^{n-2}, 3\omega^{n-1}$
        - for column C: $2\omega^1, 2\omega^2, 2\omega^3, 2\omega^4, ..., 2\omega^{n}, 3\omega^{n}$

3. construct 2 polynomials $f, g$ representing the values in the computation trace table, Let:
  - $f'(\omega^i)=f(\omega^i)+\beta · i+\gamma$, $i$ represent index in matrix before permutation
  - $g'(\omega^i)=g(\omega^i)+\beta · \sigma(i)+\gamma$, $\sigma(i)$ represent index in matrix after permutation
4. then Prover computes $Z \in F_{<n}[X]$, s.t $Z(\omega)=1$; and for $i\in \{2,...,n\}$ $$Z(\omega^i) = \prod_{1\le j<i} \frac{f'(\omega^j)}{g'(\omega^j)}$$
5. Verifier checks if for all $a\in H$
  - $L_1(a)(Z(a)-1)=0$
  - $Z(a)f'(a) = g'(a)Z(a·\omega)$


In [ ]:
# generate two random number beta and gamma
beta = FQ(random.randint(0, p))
gamma = FQ(random.randint(0, p))
print(f'beta = {beta}, gamma = {gamma}')

beta = 25227428662100397909003723125956767219844274980029406864409905825009699272612, gamma = 2001942086730337737869128807666465678864934683001832738425705322712899080924


In [ ]:
# Construct two matrixes
# before permutation
def generate_ori_matrix(n):
  ori_matrix = []
  omega = find_nth_root_of_unity_realnum(n, p)
  for i in range(n):
    row = []
    for j in range(1,4):
      row.append(FQ(j)*(omega**i))
    ori_matrix.append(row)
  return ori_matrix

ori_matrix = generate_ori_matrix(n)
# ori_matrix

In [ ]:
# after permutation
def permutation_matrix(ori_matrix, n):
  permu_matrix = ori_matrix
  for i in range(n-1):
    permu_matrix[i+1][0], permu_matrix[i][1] = permu_matrix[i][1], permu_matrix[i+1][0]
    permu_matrix[i+1][1], permu_matrix[i][2] = permu_matrix[i][2], permu_matrix[i+1][1]
  return permu_matrix

permu_matrix = permutation_matrix(ori_matrix, n)
# permu_matrix

In [ ]:
# f, g polynomial construction
def f_g_poly(value_in_trace_table, index, beta, gamma):
  return value_in_trace_table + beta * index + gamma

In [ ]:
# calculate Z
def Z_poly_evals(trace_table, ori_matrix, permu_matrix, beta, gamma, n, p):
  Z_values = [FQ(1)]
  omega = find_nth_root_of_unity_realnum(n, p)

  # compute Z in lagrange form according to its definition
  for i in range(n):
    Z_values.append(
        Z_values[-1]
        * f_g_poly(trace_table[i][0], ori_matrix[i][0], beta, gamma)
        * f_g_poly(trace_table[i][1], ori_matrix[i][1], beta, gamma)
        * f_g_poly(trace_table[i][2], ori_matrix[i][2], beta, gamma)
        / f_g_poly(trace_table[i][0], permu_matrix[i][0], beta, gamma)
        / f_g_poly(trace_table[i][1], permu_matrix[i][1], beta, gamma)
        / f_g_poly(trace_table[i][2], permu_matrix[i][2], beta, gamma)
    )
  assert Z_values.pop() == 1

  # check if Z is calculated correctly
  for i in range(n):
    assert(
        f_g_poly(trace_table[i][0], ori_matrix[i][0], beta, gamma)
        * f_g_poly(trace_table[i][1], ori_matrix[i][1], beta, gamma)
        * f_g_poly(trace_table[i][2], ori_matrix[i][2], beta, gamma)
    ) * Z_values[i] - (
        f_g_poly(trace_table[i][0], permu_matrix[i][0], beta, gamma)
        * f_g_poly(trace_table[i][1], permu_matrix[i][1], beta, gamma)
        * f_g_poly(trace_table[i][2], permu_matrix[i][2], beta, gamma)
    ) * Z_values[
        (i+1) % n
    ]== 0

  # # transfer Z polynomial to coefficient form
  # z_coefs = IFFT(Z_values, p)

  return Z_values

z_poly_evals = Z_poly_evals(trace_table, ori_matrix, permu_matrix, beta, gamma, n, p)

In [ ]:
# wiring constraints
# construct the polynomial we need to check: 𝑍(𝑎)𝑓′(𝑎) - 𝑔′(𝑎)𝑍(𝑎·𝜔)
def wiring_constraint_poly_permuation(n, p, trace_table, ori_matrix, beta, gamma, z_poly_evals):

  f_poly_evals = []
  g_poly_evals = []

  for i in range(n):
    f_poly_evals.append(
        f_g_poly(trace_table[i][0], ori_matrix[i][0], beta, gamma)
        * f_g_poly(trace_table[i][1], ori_matrix[i][1], beta, gamma)
        * f_g_poly(trace_table[i][2], ori_matrix[i][2], beta, gamma))
    g_poly_evals.append(
        f_g_poly(trace_table[i][0], permu_matrix[i][0], beta, gamma)
        * f_g_poly(trace_table[i][1], permu_matrix[i][1], beta, gamma)
        * f_g_poly(trace_table[i][2], permu_matrix[i][2], beta, gamma))
    z_poly_evals_xomega = z_poly_evals[1:]+[z_poly_evals[0]]

  f_poly_coefs = IFFT(f_poly_evals, p)
  g_poly_coefs = IFFT(g_poly_evals, p)
  z_poly_coefs = IFFT(z_poly_evals, p)
  z_poly_coefs_xomega = IFFT(z_poly_evals_xomega, p)

  phi_w_coefs = sub_poly(mul_poly(z_poly_coefs, f_poly_coefs), mul_poly(g_poly_coefs, z_poly_coefs_xomega))

  return phi_w_coefs

wiring_constraint_poly_permuation(n, p, trace_table, ori_matrix, beta, gamma, z_poly_evals)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]